In [1]:
#pip install mysqlclient

In [55]:
import json
from Bio import Entrez
from Bio import Medline

import os

# pubmed

In [56]:
Entrez.email = "tariqf549@gmail.com"
handle = Entrez.einfo() # or esearch, efetch, ...
record = Entrez.read(handle)
handle.close()

In [57]:
querylist = ('clinical trial[Title/Abstract]' , 'italy[Title/Abstract]' , 'netherlands[Title/Abstract]' , 'case control study' , 'epidemiology' , 
             'mortality', '(treatment[All Fields] OR drug[All Fields] OR intervention[All Fields] OR recovery[All Fields])' )

In [58]:
#myterm = MainTerm + """ AND trial[Title/Abstract] """

In [59]:
resultsfilename='results.json'

In [60]:
jsonfilename = resultsfilename
if os.path.isfile(jsonfilename):
    mainDict = json.load(open(jsonfilename))
else:
    mainDict = {}


In [61]:
def add2dict(dataresults , Q):
    for hit in dataresults:
        m1 = 'PMID' + hit
        parse_res = Medline.read(m1.split('\n'))
        
        PMID = parse_res['PMID']
        if PMID in mainDict.keys():
            print('PMID exists')
            if Q not in mainDict[PMID]['Tag']:
                mainDict[PMID]['Tag']+';'+Q
            continue
        else:
            Title = parse_res['TI']
            dateP = parse_res['DP']
            Tag = Q
            if 'JT' in parse_res.keys():
                JournalName  = parse_res['JT']
            else: JournalName = ''

            if 'LR' in parse_res.keys():
                dateMod  = parse_res['LR']
            else: dateMod = ''

            if 'AB' in parse_res.keys():
                Abstract = parse_res['AB']
            else: Abstract = 'NA'
            Link= 'https://www.ncbi.nlm.nih.gov/pubmed/{}'.format(PMID)   


            mainDict[PMID] = {'PMID': PMID, 'Title':Title ,
                                       'JournalName':JournalName ,
                                        'Modification Date':dateMod , 
                                       'Publication Date':dateP , 
                                       'Abstract':Abstract , 
                                       'Link':Link,
                                       'Tag':Tag
                                         
                             }


In [62]:
def search_function (MyTerms, startD , endD):

    Entrez.email = "tariqf549@gmail.com"
    MainTerm = """(((("coronavirus"[MeSH Terms] OR "coronavirus"[All Fields]) AND ("COVID-19"[All Fields] OR "severe acute respiratory syndrome coronavirus 2"[Supplementary Concept] OR "severe acute respiratory syndrome coronavirus 2"[All Fields] OR "2019-nCoV"[All Fields] OR "SARS-CoV-2"[All Fields] OR "2019nCoV"[All Fields] ))))"""
    
    #MainTerm = '"COVID-19"[All Fields]'
    DateRange = '"{}"[PDat] : "{}"[PDat]'.format(startD , endD)
    myterm = MainTerm + ' AND ' + MyTerms + ' AND ' + DateRange
    print(myterm)
    search_results = Entrez.read(
        Entrez.esearch(
            db="pubmed", term=myterm,  datetype="pdat", usehistory="y" , sort = 'relevance' 
        )
    )
    count = int(search_results["Count"])
    print("Found %i results" % count)

    batch_size = 10
    out_handle = open("corona_{}_papers.txt".format(MyTerms.split('[')[0]), "w")
    for start in range(0, count, batch_size):
        end = min(count, start + batch_size)
        print("Going to download record %i to %i" % (start + 1, end))
        fetch_handle = Entrez.efetch(
            db="pubmed",
            rettype="medline",
            retmode="text",
            retstart=start,
            retmax=batch_size,
            webenv=search_results["WebEnv"],
            query_key=search_results["QueryKey"],
        )
        data = fetch_handle.read()
        
        dataresults = data.split('\nPMID')[1:]
        add2dict(dataresults , MyTerms)
        
        fetch_handle.close()
        out_handle.write(data)
    out_handle.close()

In [63]:
querylist[0]

'clinical trial[Title/Abstract]'

In [54]:
#for X in querylist:
#    search_function(X , '2020/12/26' , '2020/03/27' )

(((("coronavirus"[MeSH Terms] OR "coronavirus"[All Fields]) AND ("COVID-19"[All Fields] OR "severe acute respiratory syndrome coronavirus 2"[Supplementary Concept] OR "severe acute respiratory syndrome coronavirus 2"[All Fields] OR "2019-nCoV"[All Fields] OR "SARS-CoV-2"[All Fields] OR "2019nCoV"[All Fields] )))) AND clinical trial[Title/Abstract] AND "2019/12/01"[PDat] : "2020/01/01"[PDat]
Found 0 results
(((("coronavirus"[MeSH Terms] OR "coronavirus"[All Fields]) AND ("COVID-19"[All Fields] OR "severe acute respiratory syndrome coronavirus 2"[Supplementary Concept] OR "severe acute respiratory syndrome coronavirus 2"[All Fields] OR "2019-nCoV"[All Fields] OR "SARS-CoV-2"[All Fields] OR "2019nCoV"[All Fields] )))) AND italy[Title/Abstract] AND "2019/12/01"[PDat] : "2020/01/01"[PDat]
Found 0 results
(((("coronavirus"[MeSH Terms] OR "coronavirus"[All Fields]) AND ("COVID-19"[All Fields] OR "severe acute respiratory syndrome coronavirus 2"[Supplementary Concept] OR "severe acute respirat

TypeError: can only concatenate list (not "str") to list

In [42]:
with open(resultsfilename , 'w') as fp:   
    Output=[]
    for Key,Item in mainDict.items():
        Item['Tag'] = ';'.join(Item['Tag'])
        Output.append(Item)
    json.dump(Output, fp)

clinical trial[Title/Abstract];epidemiology;(treatment[All Fields] OR drug[All Fields] OR intervention[All Fields] OR recovery[All Fields])
clinical trial[Title/Abstract];(treatment[All Fields] OR drug[All Fields] OR intervention[All Fields] OR recovery[All Fields])
clinical trial[Title/Abstract];mortality;(treatment[All Fields] OR drug[All Fields] OR intervention[All Fields] OR recovery[All Fields])
clinical trial[Title/Abstract];(treatment[All Fields] OR drug[All Fields] OR intervention[All Fields] OR recovery[All Fields])
clinical trial[Title/Abstract];(treatment[All Fields] OR drug[All Fields] OR intervention[All Fields] OR recovery[All Fields])
italy[Title/Abstract];epidemiology
italy[Title/Abstract];epidemiology;(treatment[All Fields] OR drug[All Fields] OR intervention[All Fields] OR recovery[All Fields])
italy[Title/Abstract];epidemiology;mortality
italy[Title/Abstract];(treatment[All Fields] OR drug[All Fields] OR intervention[All Fields] OR recovery[All Fields])
italy[Title/A

In [51]:
'italy[Title/Abstract]' not in 'italy[Title/Abstract];epidemiology;mortality'

False

In [47]:
headerslist = []
for H in ('PMID',
          'Title',
          'Link',
            'JournalName',
            'Modification Date',
            'Publication Date',
            'Abstract',
            'Tag'):
    print(H)
    headers = '=ImportJSON("https://raw.githubusercontent.com/tofaquih/coronaPubGet/master/results.json", "/{}", "noInherit, noTruncate")'.format(H)
    headerslist.append(headers)
    
headerslist

PMID
Title
Link
JournalName
Modification Date
Publication Date
Abstract
Tag


['=ImportJSON("https://raw.githubusercontent.com/tofaquih/coronaPubGet/master/results.json", "/PMID", "noInherit, noTruncate")',
 '=ImportJSON("https://raw.githubusercontent.com/tofaquih/coronaPubGet/master/results.json", "/Title", "noInherit, noTruncate")',
 '=ImportJSON("https://raw.githubusercontent.com/tofaquih/coronaPubGet/master/results.json", "/Link", "noInherit, noTruncate")',
 '=ImportJSON("https://raw.githubusercontent.com/tofaquih/coronaPubGet/master/results.json", "/JournalName", "noInherit, noTruncate")',
 '=ImportJSON("https://raw.githubusercontent.com/tofaquih/coronaPubGet/master/results.json", "/Modification Date", "noInherit, noTruncate")',
 '=ImportJSON("https://raw.githubusercontent.com/tofaquih/coronaPubGet/master/results.json", "/Publication Date", "noInherit, noTruncate")',
 '=ImportJSON("https://raw.githubusercontent.com/tofaquih/coronaPubGet/master/results.json", "/Abstract", "noInherit, noTruncate")',
 '=ImportJSON("https://raw.githubusercontent.com/tofaquih/co

with open('ct.json' , 'w') as fp:
    json.dump(mainDict['clinical trial[Title/Abstract]'], fp)
    

In [27]:
with open(resultsfilename , 'w') as fp:
    json.dump(mainDict, fp)

In [ ]:
#pip install MySQL-python

In [10]:
import mysql.connector
mydb = mysql.connector.connect(
  host="localhost",
  user="yourusername",
  passwd="yourpassword"
)
mycursor = mydb.cursor()

mycursor.execute("CREATE DATABASE mydatabase")
print(mydb)

ModuleNotFoundError: No module named 'mysql'

In [ ]:
print("\033[44;33mHello World!\033[m")


# google

In [ ]:
import requests
from bs4 import BeautifulSoup

query = '"covid-19"  italy'
url =  'https://scholar.google.com/scholar?start=0&q='+ query + '&hl=en&scisbd=1&as_sdt=1,5&as_vis=1&ie=UTF-8&oe=UTF-8&hl=en&btnG=Search'

content = requests.get(url).text
page = BeautifulSoup(content, 'html')
results = []
for entry in page.find_all("h3", attrs={"class": "gs_rt"}):
    results.append({"title": entry.a.text, "url": entry.a['href']})

In [ ]:
for entry in page.find_all(attrs={"class": "gs_rs"}):
    maintext = entry.get_text()
    D = maintext.split(' ')[0]
    print(D)
    print(entry.get_text())
    print()

In [ ]:
results[0]